In [1]:
import tensorflow as tf
import pathlib
import os

from tensorflow.keras.applications import VGG19     # type: ignore
from tensorflow.keras.layers import  Dense, Flatten # type: ignore
from tensorflow.keras.models import Model           # type: ignore

import numpy as np

2024-10-17 01:20:32.857865: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-17 01:20:32.869787: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-17 01:20:32.884035: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-17 01:20:32.887829: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-17 01:20:32.898540: I tensorflow/core/platform/cpu_feature_guar

In [2]:
localPath = pathlib.Path().resolve()

train_data_path = pathlib.Path(localPath ,"../Dataset/TRAIN")
test_data_path = pathlib.Path(localPath ,"../Dataset/TEST")
val_data_path = pathlib.Path(localPath ,"../Dataset/VALIDATION")

class_names = np.array(sorted([item.name for item in train_data_path.glob('*') if item.name != "LICENSE.txt"]))

AUTOTUNE = tf.data.AUTOTUNE
class_names = np.array(sorted([item.name for item in train_data_path.glob('*') if item.name != "LICENSE.txt"]))


img_size = 224
BATCH_SIZE = 64

In [3]:
def get_label(file_path):
    # Convert the path to a list of path components
    parts = tf.strings.split(file_path, os.path.sep)
    # The second to last is the class-directory
    one_hot = parts[-2] == class_names
    # Integer encode the label
    return tf.cast(one_hot, tf.int64)

def decode_img(img):
    # Convert the compressed string to a 3D uint8 tensor
    img = tf.io.decode_jpeg(img, channels=3)
    # Resize the image to the desired size
    return tf.image.resize(img, [img_size, img_size])

def process_path(file_path):
    label = get_label(file_path)
    # Load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img #, label

def make_dataset(path):
    ds = tf.data.Dataset.list_files(str(path/'*/*'), shuffle=True)
    ds = ds.map(process_path, num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE)
    return ds

def get_labels(path):
    ds = tf.data.Dataset.list_files(str(path/'*/*'), shuffle=True)
    ds = ds.map(get_label, num_parallel_calls=AUTOTUNE).batch(15194)
    return ds

In [4]:
train_datasets = make_dataset(train_data_path)
test_datasets = make_dataset(test_data_path)
validation_data, = make_dataset(val_data_path)

base_model = VGG19(weights='imagenet', include_top=True)

train_datasets_vector = base_model.predict(train_datasets)
test_datasets_vector = base_model.predict(test_datasets)
validation_datasets_vector = base_model.predict(validation_data)

I0000 00:00:1729128034.616888   30749 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729128034.622007   30749 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729128034.622049   30749 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729128034.623825   30749 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729128034.623855   30749 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

  2/238 ━━━━━━━━━━━━━━━━━━━━ 23s 101ms/step

I0000 00:00:1729128050.972735   30862 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


238/238 ━━━━━━━━━━━━━━━━━━━━ 41s 127ms/step
37/38 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step

2024-10-17 01:21:27.624682: W external/local_tsl/tsl/framework/bfc_allocator.cc:291] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.46GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


38/38 ━━━━━━━━━━━━━━━━━━━━ 10s 271ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


In [6]:
train_labels = list(get_labels(train_data_path).as_numpy_iterator())
test_labels = list(get_labels(test_data_path).as_numpy_iterator())
validation_labels = list(get_labels(val_data_path).as_numpy_iterator())

2024-10-17 01:22:50.113120: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-10-17 01:22:50.727825: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [7]:
train_datasets_vector = np.concatenate((train_datasets_vector,train_labels[0]),axis=1)
test_datasets_vector = np.concatenate((test_datasets_vector,test_labels[0]),axis=1)
validation_datasets_vector = np.concatenate((validation_datasets_vector,validation_labels[0]),axis=1)

In [8]:
np.save('../DatasetVGG19/train', train_datasets_vector)
np.save('../DatasetVGG19/test', test_datasets_vector)
np.save('../DatasetVGG19/validation', validation_datasets_vector)